In [115]:
import jax
import jax.numpy as np
import pandas as pd
import itertools

Forward preprocessing: (from other algo) -> (csv)
$$
\mathcal{D}: \text{csv} \rightarrow \mathbb{R}^7
$$

In [116]:
columnNames = ["track", "time", "note_is_played"
                            , "channel", "notes", "velocity"]

df = pd.read_csv('midi/drumDemo.csv', skipfooter=2, skiprows=8, engine='python', names= columnNames)
print(df.head())

track = (df["track"].values)
time = (df["time"].values)
note_is_played = (df["note_is_played"].values)
channel = df["channel"].values
notes = df["notes"].values
velocity = df["velocity"].values

"""
print(track)
print(time)
print(note_is_played)
print(channel)
print(notes)
print(velocity)
"""

h = {}
h[36] = 0
h[38] = 1
h[41] = 2
h[42] = 3
h[43] = 4
h[45] = 5
h[82] = 6

h_inv = {}
h_inv[0] = 36
h_inv[1] = 38
h_inv[2] = 41
h_inv[3] = 42
h_inv[4] = 43
h_inv[5] = 45
h_inv[6] = 82

print()

u = list(itertools.repeat(np.zeros(7), max(time)+1))

for i in range(len(notes)):
    v = [0]*7
    v[h[notes[i]]] = velocity[i] / 127
    if (np.any(u[time[i]])):
        u[time[i]] = u[time[i]] + np.array(v)
    else:
        u[time[i]] = np.array(v)

print(u[236])

   track  time note_is_played  channel  notes  velocity
0      2     0      Note_on_c        9     42        96
1      2     0      Note_on_c        9     36        84
2      2    29      Note_on_c        9     36         0
3      2    29      Note_on_c        9     42         0
4      2   236      Note_on_c        9     38        61

[0.         0.48031497 0.         0.5590551  0.         0.
 0.        ]


Backward preprocessing
$$
\mathcal{D}^{-1} : \mathbb{R}^7 \rightarrow \text{csv}
$$

In [117]:
y = u
dfo = pd.DataFrame([], columns=columnNames)


#dfo["track"] = track
#dfo["note_is_played"] = note_is_played
#dfo["channel"] = channel
time_new = []
notes_new = []
velocity_new = []


for n in range(len(y)):
    if (np.any(y[n])):
        yn = y[n]
        for i in range(len(yn)):
            v = (int)(yn[i] * 127)
            if (v > 0):
                time_new.append(n)
                velocity_new.append(v)
                notes_new.append(h_inv[i])

track_new = [track[0]]*len(time_new)
note_is_played_new = [note_is_played[0]]*len(time_new)
channel_new = [channel[0]]*len(time_new)

dfo["track"] = track_new
dfo["note_is_played"] = note_is_played_new
dfo["channel"] = channel_new
dfo["time"] = time_new
dfo["velocity"] = velocity_new
dfo["notes"] = notes_new

dfo.to_csv("new.csv", index = False, header = False)

